In [226]:
import pandas as pd
import numpy as np
import os
import sys
import re
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
# Read cosmic/TCGA data 
cosmic_GRCh37 = pd.read_table("../raw_data/CosmicCompleteGeneExpression_GRCh37.tsv")

In [10]:
# Read GTF file 
ens69_gtf = pd.read_table("../raw_data/Homo_sapiens.GRCh37.69.gtf", header = None, sep = '\t')
ens69_gtf.columns = ['chr', 'rna_type', 'transcript_region', 'start', 'end', '5', 'strand', '7', 'transcript_info']

In [11]:
# SUMMARY STATISTICS 
## Non-normal distribution stats
### Median Z 
cosmic_GRCh37_median = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].median()
cosmic_GRCh37_median['GENE_NAME'] = cosmic_GRCh37_median.index
### IQR Z
cosmic_GRCh37_iqr = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].quantile([.25, .5, .75]).unstack()
cosmic_GRCh37_iqr['GENE_NAME'] = cosmic_GRCh37_iqr.index
cols = ['Q1', 'Q2', 'Q3', 'gene_name']
cosmic_GRCh37_iqr.columns = cols
cosmic_GRCh37_iqr = cosmic_GRCh37_iqr.reset_index(drop = True)
cosmic_GRCh37_iqr['IQR'] = cosmic_GRCh37_iqr['Q3'] - cosmic_GRCh37_iqr['Q1']
cosmic_GRCh37_iqr['lower_1.5IQR'] = cosmic_GRCh37_iqr['Q1'] - 1.5*cosmic_GRCh37_iqr['IQR']
cosmic_GRCh37_iqr['upper_1.5IQR'] = cosmic_GRCh37_iqr['Q3'] + 1.5*cosmic_GRCh37_iqr['IQR']

## Normal stats - Mean, sd, etc
cosmic_GRCh37_summary = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].describe()
cosmic_GRCh37_summary['gene_name'] = cosmic_GRCh37_summary.index
cosmic_GRCh37_summary = cosmic_GRCh37_summary.reset_index(drop = True)
cols= ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'gene_name']
cosmic_GRCh37_summary.columns = cols

## Join non-normal and normal stats 
cosmic_GRCh37_summary_full = pd.merge(cosmic_GRCh37_summary,cosmic_GRCh37_iqr, how = 'inner') 

In [13]:
cosmic_GRCh37_summary_full.to_csv("../tables_output/cosmic_GRCh37_summary_full.tsv", sep = '\t', index = False)

In [14]:
cosmic_GRCh37_summary_full.columns

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'gene_name',
       'Q1', 'Q2', 'Q3', 'IQR', 'lower_1.5IQR', 'upper_1.5IQR',
       'Q2_Z_positive'],
      dtype='object')

In [15]:
# TRANSFORM Z_SCORE DISTRIBUTION VALUES  
## Find minimum (most negative) value
min_median_Z_SCORE = min(cosmic_GRCh37_summary_full['Q2'])
## Make all median (Q2) Z_SCORE values positive 
cosmic_GRCh37_summary_full['Q2_Z_positive'] = cosmic_GRCh37_summary_full['Q2'] - (min_median_Z_SCORE)
## Find total Q2_Z_positive (to normalize)
total_median_Z_SCORE = sum(cosmic_GRCh37_summary_full['Q2_Z_positive'])

## long_cols_pro = ['Locus', 'Transcript_ID', 'Coding', 'Length', 'Expressed Fraction', 'Expressed Number', 'Library Fraction', 'Library Number', 'Sequenced Fraction', 'Sequenced Number', 'Covered Fraction', 'Chi Square', 'Coefficient of Variation']


In [17]:
# FUNCTIONS 
# Define function tidy_split()
def tidy_split(df, column, sep=';', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.
    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row
    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df.reset_index(drop=True)

In [19]:
info_data = ens69_gtf['transcript_info'].str.split(';', expand=True)
info_data = info_data.iloc[:, :7]
info_data.columns = ['gene_id', 'transcript_id', 'exon_number', 'gene_name', 
                     'gene_biotype', 'transcript_name', 'exon_id']
info_data = info_data.fillna('exon id "none"')
info_data['exon_id'] = info_data['exon_id'].replace('', 'exon id "none"')

# Define update_columns()
def update_columns(data, column):
    data[column] = data[column].map(lambda x: x.split('"')[1])
    return data
    
for i in info_data.columns:
    info_data = update_columns(info_data, i)

In [20]:
# PROCESS GTF FILE 
## Ens69 GTF data processing 
ens69_gtf_expand = pd.concat([ens69_gtf, info_data], axis=1)
ens69_gtf_expand['chr'] = ens69_gtf_expand['chr'].astype(str)

chromosomes_lst = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 
                   '10', '11', '12', '13', '14', '15', '16', '17', 
                   '18', '19', '20', '21', 'X', 'Y']

ens69_gtf_chr = ens69_gtf_expand[ens69_gtf_expand['chr'].isin(chromosomes_lst )]

In [22]:
# Get real start and end of transcripts (not exon level)
## START
transcripts_min = ens69_gtf_chr.groupby(['transcript_id'])[['start']].first().drop_duplicates()
transcripts_min = transcripts_min.dropna()
transcripts_min['transcript_id'] = transcripts_min.index
transcripts_min = transcripts_min.dropna()
transcripts_min = transcripts_min.reset_index(drop = True)
## END 
transcripts_max = ens69_gtf_chr.groupby(['transcript_id'])[['end']].max().drop_duplicates()
transcripts_max = transcripts_max.dropna()
transcripts_max['transcript_id'] = transcripts_max.index 
transcripts_max = transcripts_max.dropna()
transcripts_max = transcripts_max.reset_index(drop  = True)

## JOIN START-END
transcripts_start_end = pd.merge(transcripts_min, transcripts_max, how= 'left')
transcripts_start_end = transcripts_start_end.rename(columns={'start': 'tx_start', 'end': 'tx_end'})

In [25]:
## SUBSET GTF 
ens69_gtf_chr_subset = ens69_gtf_chr[['chr', 'gene_name', 'strand', 'transcript_id', 'gene_id', 'gene_biotype']]

In [26]:
## EXPAND GTF START-END DF
transcripts_start_end_genes = pd.merge(transcripts_start_end, ens69_gtf_chr_subset, how = 'left')
transcripts_start_end_genes = transcripts_start_end_genes.dropna()

In [34]:
## Ensure transcript start and end are integers
transcripts_start_end_genes['tx_end'] = transcripts_start_end_genes['tx_end'].astype(int)
transcripts_start_end_genes['tx_start'] = transcripts_start_end_genes['tx_start'].astype(int)
transcripts_start_end_genes['tx_len'] = transcripts_start_end_genes['tx_end'] - transcripts_start_end_genes['tx_start']

In [35]:

## Sort data, reset index 
transcripts_start_end_genes = transcripts_start_end_genes.drop_duplicates()
transcripts_start_end_genes = transcripts_start_end_genes.sort_values(['chr','tx_start',  'gene_id' , 'gene_name'])
transcripts_start_end_genes.reset_index(inplace = True, drop = True)


In [36]:
# Compute proportion of the full gene length for each transcript 
transcripts_start_end_genes['sum_tx_len'] = transcripts_start_end_genes.groupby('gene_name')['tx_len'].transform('sum')
transcripts_start_end_genes['tx_proportion'] = transcripts_start_end_genes['tx_len'] / transcripts_start_end_genes['sum_tx_len']

In [39]:
# Rename column expression 
cosmic_GRCh37_iqr = cosmic_GRCh37_iqr.rename(columns = {'GENE_NAME':'gene_name'})
cosmic_GRCh37_summary_full = cosmic_GRCh37_summary_full.rename(columns = {'GENE_NAME':'gene_name'})

In [83]:
# Create expressed fraction and expressed number columns 
## Count the number of transcripts and assume equal contribution of RNA from each transcript not accounting for length
cosmic_GRCh37_summary_full['count_transcripts'] = cosmic_GRCh37_summary_full.groupby('gene_name')['gene_name'].transform('count')

## * PRO Expressed Fraction col *
cosmic_GRCh37_summary_full['Expressed Fraction'] = cosmic_GRCh37_summary_full['Q2_Z_positive']/(total_median_Z_SCORE) 
# Adjust for number of transcripts 
cosmic_GRCh37_summary_full['Expressed Fraction - Transcript'] = cosmic_GRCh37_summary_full['Expressed Fraction']/cosmic_GRCh37_summary_full['count_transcripts']

## * PRO Expressed Number col *
cosmic_GRCh37_summary_full['Expressed Number'] = cosmic_GRCh37_summary_full['Expressed Fraction'] * 100000
# Adjust for number of transcripts 
cosmic_GRCh37_summary_full['Expressed Number - Transcript'] = cosmic_GRCh37_summary_full['Expressed Number']/cosmic_GRCh37_summary_full['count_transcripts']

# Round to integer for Expressed Number 
cosmic_GRCh37_summary_full = cosmic_GRCh37_summary_full.round({'Expressed Number - Transcript':0})

In [84]:
## MERGE EXPRESSION + GTF INFORMATION 
exp_gtf = pd.merge(cosmic_GRCh37_summary_full, transcripts_start_end_genes, how = 'inner')

## Count the number of transcripts and assume equal contribution of RNA from each transcript not accounting for length
exp_gtf['count_transcripts'] = exp_gtf.groupby('gene_name')['gene_name'].transform('count')

## Strand system switch 
exp_gtf['strand_WC'] = pd.Series(exp_gtf['strand']).str.replace('+', 'W')
exp_gtf['strand_WC'] = pd.Series(exp_gtf['strand']).str.replace('-', 'C')

## * PRO Locus col *
exp_gtf['Locus'] = exp_gtf['chr'].astype(str) + ":" + exp_gtf['tx_start'].astype(str) + "-" + exp_gtf['tx_end'].astype(str) + exp_gtf['strand_WC'].astype(str)


In [85]:
exp_gtf.head(2)

,count,mean,std,min,25%,50%,75%,max,gene_name,Q1,...,chr,strand,gene_id,gene_biotype,len,tx_len,sum_tx_len,tx_proportion,strand_WC,Locus
0,9144.0,0.334597,25.622720,-1.570,-0.60125,-0.283,0.242,2444.419,A1BG,-0.60125,...,19,-,ENSG00000121410,protein_coding,95,95,95,1.000000,C,19:58864770-58864865C
1,9144.0,0.201225,6.548947,-2.224,-0.26100,-0.203,-0.096,332.616,A1CF,-0.26100,...,10,-,ENSG00000148584,protein_coding,73,73,310,0.235484,C,10:52588148-52588221C


In [86]:
## Function to determine if transcript CDS (coding) or NC (noncoding)
def determine_coding(x):
    if x=='protein_coding':
        return('CDS')
    else:
        return('NC')

## * PRO Coding col *     
exp_gtf['Coding'] = exp_gtf['gene_biotype'].map(determine_coding)    

## * PRO Length col * 
exp_gtf['Length'] = exp_gtf['tx_end'] - exp_gtf['tx_start']


In [87]:
# FINAL PRO 
PRO_final = exp_gtf[['Locus', 'transcript_id', 'Coding', 'Length', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']]

In [88]:
## Remove all transcripts < 10 nt 
PRO_final_clean = PRO_final[PRO_final['Length'] >= 10]

In [89]:
PRO_final_clean.head()

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
0,19:58864770-58864865C,ENST00000263100,CDS,95,0.000050,5.0
1,10:52588148-52588221C,ENST00000493415,CDS,73,0.000055,6.0
2,10:52619602-52619745C,ENST00000373993,CDS,143,0.000055,6.0
3,10:52645341-52645435C,ENST00000282641,CDS,94,0.000055,6.0
4,13:101185541-101185907C,ENST00000464500,CDS,366,0.000049,5.0


In [113]:
print(PRO_final_clean.shape)
print(len(PRO_final_clean['transcript_id'].unique()))

(85524, 6)
85524


In [148]:
PRO_final_clean.head(2)

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
0,19:58864770-58864865C,ENST00000263100,CDS,95,0.000050,5
1,10:52588148-52588221C,ENST00000493415,CDS,73,0.000055,6


In [90]:
PRO_final_clean['Expressed Number - Transcript'] = (PRO_final_clean['Expressed Number - Transcript']).astype(int)
# Compute total number of molecules 
total_N_MOLECULES = sum(PRO_final_clean['Expressed Number - Transcript']).astype(int)

In [91]:
total_N_MOLECULES

471461

In [95]:
pro_file_name = 'TCGA_dataEXP_median_N_MOLECULES_' + total_N_MOLECULES.astype(str) + ".tsv"
pro_file_name

'TCGA_dataEXP_median_N_MOLECULES_471461.tsv'

In [96]:
PRO_final_clean.to_csv(('../tables_output/' + pro_file_name), sep = '\t', index = False, header = None)

In [164]:
## SUBSET CHR12 and CHR21
exp_gtf_chr12_chr21 = exp_gtf[exp_gtf['chr'].isin(['12', '21'])] 
PRO_final_clean_chr12_chr21 = exp_gtf_chr12_chr21[['Locus', 'transcript_id', 'Coding', 'Length', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']]


In [165]:
PRO_final_clean_chr12_chr21.head()

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
9,12:9221336-9221551C,ENST00000495442,CDS,215,0.000049,5.0
10,12:9232870-9232994C,ENST00000542567,CDS,124,0.000049,5.0
11,12:9242952-9243126C,ENST00000462568,CDS,174,0.000049,5.0
12,12:9243797-9244008C,ENST00000543436,CDS,211,0.000049,5.0
13,12:9254043-9254152C,ENST00000472360,CDS,109,0.000049,5.0


In [181]:
## THEN RE-COMPUTE TOTAL READS 
total_N_MOLECULES = sum(PRO_final_clean_chr12_chr21['Expressed Number - Transcript']).astype(int)

## PROPORTION
PRO_final_clean_chr12_chr21['Expressed Fraction - Transcript'] = PRO_final_clean_chr12_chr21['Expressed Number - Transcript']/total_N_MOLECULES

## INTEGER
PRO_final_clean_chr12_chr21['Expressed Number - Transcript'] = PRO_final_clean_chr12_chr21['Expressed Number - Transcript'].astype(int)


In [182]:
PRO_final_clean_chr12_chr21.head(2)

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
9,12:9221336-9221551C,ENST00000495442,CDS,215,0.000137,5
10,12:9232870-9232994C,ENST00000542567,CDS,124,0.000137,5


In [183]:
pro_file_name = 'TCGA_dataEXP_median_N_MOLECULES_' + total_N_MOLECULES.astype(str) + ".tsv"
pro_file_name

'TCGA_dataEXP_median_N_MOLECULES_36600.tsv'

In [184]:
PRO_final_clean_chr12_chr21.to_csv(('../tables_output/' + "chr12_chr21_" + pro_file_name), sep = '\t', index = False, header = None)

--- 

## PRO file generated by Flux simulator 

In [169]:
pro_man = pd.read_table("../tables_output/12_21_test.pro", sep = '\t', header = None)

In [170]:
pro_man.column = ['Locus', 'transcript_id', 'Coding', 'Length', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']

In [185]:
pro_man_v2 = pro_man[[0, 1, 2, 3, 4, 5]]
pro_man_v2.columns = ['Locus', 'transcript_id', 'Coding', 'Length', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']

In [186]:
pro_man_v2.head(2)

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
0,12:67607-69138W,ENST00000504074,NC,1187,0.000012,122
1,12:67607-69138W,ENST00000546223,NC,1199,0.000026,264


In [189]:
PRO_MLT = PRO_final_clean_chr12_chr21[['transcript_id', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']]

In [190]:
only_man = [x for x in pro_man_v2['transcript_id'].unique() if x not in PRO_MLT['transcript_id'].unique()]

In [191]:
# Filter for only in 
ens69_gtf_only_man = pro_man_v2[pro_man_v2['transcript_id'].isin(only_man)]


In [193]:
ens69_gtf_only_man.shape

(7085, 6)

In [197]:
MERGE_PRO_TCGA = pd.concat([ens69_gtf_only_man, PRO_MLT])

In [198]:
MERGE_PRO_TCGA.head(2)

,Coding,Expressed Fraction - Transcript,Expressed Number - Transcript,Length,Locus,transcript_id
0,NC,0.000012,122,1187.0,12:67607-69138W,ENST00000504074
1,NC,0.000026,264,1199.0,12:67607-69138W,ENST00000546223


In [199]:
MERGE_PRO_TCGA.shape

(13584, 6)

In [200]:
final_PRO_TCGA = MERGE_PRO_TCGA[['Locus', 'transcript_id', 'Coding', 'Length', 'Expressed Fraction - Transcript', 'Expressed Number - Transcript']]

In [210]:
final_PRO_TCGA = final_PRO_TCGA.drop_duplicates()

In [211]:
total_N_MOLECULES = sum(final_PRO_TCGA['Expressed Number - Transcript'])

final_PRO_TCGA['Expressed Fraction - Transcript'] = final_PRO_TCGA['Expressed Number - Transcript']/total_N_MOLECULES

In [212]:
final_PRO_TCGA.head(2)

,Locus,transcript_id,Coding,Length,Expressed Fraction - Transcript,Expressed Number - Transcript
0,12:67607-69138W,ENST00000504074,NC,1187.0,0.000022,122
1,12:67607-69138W,ENST00000546223,NC,1199.0,0.000048,264


In [213]:
sum(final_PRO_TCGA['Expressed Fraction - Transcript'])

0.9999999999999768

In [228]:
print(sum(final_PRO_TCGA['Expressed Number - Transcript']))

539668


In [223]:
## SMALL
small_final_PRO_TCGA = final_PRO_TCGA
small_final_PRO_TCGA['Expressed Number - Transcript'] = small_final_PRO_TCGA['Expressed Number - Transcript'] / 10
small_final_PRO_TCGA['Expressed Number - Transcript'] = small_final_PRO_TCGA['Expressed Number - Transcript'].astype(int)

In [224]:
total_N_MOLECULES = sum(small_final_PRO_TCGA['Expressed Number - Transcript'])
total_N_MOLECULES

539668

In [225]:
final_PRO_TCGA.to_csv(('../tables_output/final_PRO_TCGA_expanded_N_MOLECULES_' + total_N_MOLECULES.astype(str) + '.tsv'), sep = '\t', index = False)

In [227]:
small_final_PRO_TCGA.to_csv(('../tables_output/final_SMALL_PRO_TCGA_expanded_N_MOLECULES_' + total_N_MOLECULES.astype(str) + '.tsv'), sep = '\t', index = False)

# Compare Flux simulator reads RNA vs the input TCGA expression PRO 

In [229]:
small_final_PRO_TCGA.to_csv(('../tables_output/final_SMALL_PRO_TCGA_expanded_N_MOLECULES_' + total_N_MOLECULES.astype(str) + '.PRO'), sep = '\t', index = False)

---

In [ ]:
## Subset the GTF to only contain the same transcripts than expression data ???

---

# Save expression files per chromosome

In [ ]:
for one_chr in chromosomes_lst:
    